In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import calendar

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
orders = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/olist_orders_dataset.csv")
order_payments = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/olist_order_payments_dataset.csv")
order_items = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/olist_order_items_dataset.csv")
products = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/olist_products_dataset.csv")
customers = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/olist_customers_dataset.csv")
geolocation = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/olist_geolocation_dataset.csv")
order_reviews = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/olist_order_reviews_dataset.csv")

In [ ]:
order_payments

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


배송 완료된 것들만 추출

In [ ]:
orders.groupby('order_status').count()

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
order_status,,,,,,,
approved,2,2,2,2,0,0,2
canceled,625,625,625,484,75,6,625
created,5,5,5,0,0,0,5
delivered,96478,96478,96478,96464,96476,96470,96478
invoiced,314,314,314,314,0,0,314
processing,301,301,301,301,0,0,301
shipped,1107,1107,1107,1107,1107,0,1107
unavailable,609,609,609,609,0,0,609


In [ ]:
orders = orders[orders.order_status == 'delivered']

In [ ]:
orders.dropna(axis=0)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


#Categorize **Purchase Time**


##Timeslots
0 - 00:00 ~ 06:00 <br>
1 - 06:00 ~ 12:00 <br>
2 - 12:00 ~ 18:00 <br>
3 - 18:00 ~ 00:00 <br>

In [ ]:
def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

In [ ]:
def time_sorting(x):
  #2018-03-08 20:57:30
  x1 = int(x[11:13])
  x2 = int(x[14:16])
  x3 = int(x[17:19])
  now = dt.time(x1, x2, x3)

  if time_in_range(dt.time(0, 0, 1), dt.time(6, 0, 0), now) == True:
    return 0
  elif time_in_range(dt.time(6, 0, 1), dt.time(12, 0, 0), now) == True:
    return 1
  elif time_in_range(dt.time(12, 0, 1), dt.time(18, 0, 0), now) == True:
    return 2
  elif time_in_range(dt.time(18, 0, 1), dt.time(0, 0, 0), now) == True:
    return 3

In [ ]:
# Test Code
time_sorting('2018-03-08 20:57:30')

3

In [ ]:
 orders['order_timerange'] = orders.apply(lambda x: time_sorting(x['order_purchase_timestamp']), axis=1)

In [ ]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_timerange
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,3
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,3
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,3
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,1
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,2
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,2
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,3


##Weekdays and ends
0 - Mon ~ Fri <br>
1 - Sat ~ Sun <br>

In [ ]:
from datetime import datetime as dt

In [ ]:
def is_weekend(d):
    now = dt.strptime(d, '%Y-%m-%d %H:%M:%S')
    return now.weekday()

In [ ]:
def is_weekend2(x):
    return x.weekday() > 4

In [ ]:
def date_sorting(x):
  #2018-03-08 20:57:30
  now = dt.strptime(x, '%Y-%m-%d %H:%M:%S')
  if is_weekend2(now) == True:
    return 1
  else:
    return 0

In [ ]:
#test
is_weekend('2018-03-10 20:57:30')

5

In [ ]:
orders['order_daytwo'] = orders.apply(lambda x: date_sorting(x['order_purchase_timestamp']), axis=1)

In [ ]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_timerange,order_daytwo
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,3,0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,3,1
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,3,0
...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,1,0
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,2,0
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,2,1
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,3,0


In [ ]:
orders['order_day'] = orders.apply(lambda x: is_weekend(x['order_purchase_timestamp']), axis=1)

In [ ]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_timerange,order_daytwo,order_day
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,0,0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,3,0,1
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,0,2
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,3,1,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,1,0,3
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,2,0,1
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,2,1,6
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,3,0,0


In [ ]:
orders.groupby('order_day').count()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_timerange,order_daytwo
order_day,,,,,,,,,,
0,15701,15701,15701,15701,15701,15701,15701,15701,15701,15701
1,15502,15502,15502,15502,15502,15502,15502,15502,15502,15502
2,15074,15074,15074,15074,15074,15074,15074,15074,15074,15074
3,14320,14320,14320,14320,14320,14320,14320,14320,14320,14320
4,13680,13680,13680,13680,13680,13680,13680,13680,13680,13680
5,10547,10547,10547,10547,10547,10547,10547,10547,10547,10547
6,11631,11631,11631,11631,11631,11631,11631,11631,11631,11631


#**Merging**

In [ ]:
import datetime as dt

In [ ]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_timerange,order_daytwo,order_day
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,0,0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,3,0,1
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,0,2
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,3,1,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,1,0,3
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,2,0,1
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,2,1,6
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,3,0,0


In [ ]:
reviews = orders.merge(order_reviews, on="order_id")

In [ ]:
reviews.to_csv("/content/gdrive/My Drive/Colab Notebooks/time_review.csv")

##Merging - Payment Type

In [ ]:
paymenttype = orders.merge(order_payments, on="order_id")

In [ ]:
paymenttype.to_csv("/content/gdrive/My Drive/Colab Notebooks/time_payment_type.csv")

##Merging - Category

In [ ]:
category = orders.merge(order_items, on="order_id")

In [ ]:
category2 = category.merge(products, on="product_id")

In [ ]:
category2
category2.to_csv("/content/gdrive/My Drive/Colab Notebooks/time_category.csv")

##Merging - Geography

In [ ]:
geolocation2 = geolocation.rename(columns={'geolocation_zip_code_prefix': 'customer_zip_code_prefix'})

In [ ]:
cust_geo = orders.merge(customers, on="customer_id")

In [ ]:
cust_geo1 = cust_geo.merge(geolocation2, on="customer_zip_code_prefix")

In [ ]:
cust_geo1.to_csv("/content/gdrive/My Drive/Colab Notebooks/time_geolocation.csv")

##Merging - RFM
Recency - orders > purchase_time <br>
Frequency - how many times did the customer buy? <br>
Money - how much money did the customer buy? <br>
https://www.kaggle.com/ceruttivini/rfm-segmentation-and-customer-analysis

In [ ]:
order_items.groupby('order_id').size().sort_values(ascending=False)

order_id
8272b63d03f5f79c56e9e4120aec44ef    21
ab14fdcfbe524636d65ee38360e22ce8    20
1b15974a0141d54e36626dca3fdc731a    20
9ef13efd6949e4573a18964dd1bbe7f5    15
428a2f660dc84138d969ccd69a0ab6d5    15
                                    ..
a6e8ad5db31e71f5f12671af561acb4a     1
a6e963c11e80432334e984ead4797a8b     1
a6e9b80a7636eb8dd592dbb3e20d0a91     1
a6e9d106235bcf1dda54253686d89e99     1
00010242fe8c5a6d1ba2dd792cb16214     1
Length: 98666, dtype: int64

In [ ]:
order_items_silver = order_items.groupby('order_id').agg({'price':sum,'freight_value':sum,'shipping_limit_date':max }).reset_index()

In [ ]:
columns_timestamp = ['order_purchase_timestamp','order_approved_at',
                     'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for column in columns_timestamp:
    orders[column] = pd.to_datetime(orders[column])

In [ ]:
orders['diff_delivery_days'] = (orders['order_estimated_delivery_date'] - orders['order_delivered_customer_date']).dt.days

In [ ]:
orders_silver = orders.merge(order_items_silver,on='order_id',how='inner')

In [ ]:
orders_customers = customers.merge(orders_silver, on='customer_id', how='inner')

In [ ]:
max_date = max(orders_customers['order_purchase_timestamp']) + dt.timedelta(days=1)

In [ ]:
rfm_data = orders_customers.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (max_date - x.max()).days,
    'customer_id':'count',
    'price':'sum',
    'order_day': 'sum',
    'order_timerange': 'sum',
}).reset_index()
rfm_data.columns =['customer_id','recency','frequency','monetary','order_day','order_timerange']

In [ ]:
rfm_data2 = rfm_data[rfm_data['order_timerange']<4]
rfm_data3 = rfm_data2[rfm_data['order_day']<2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
rfm_data3

,customer_id,recency,frequency,monetary,order_day,order_timerange
1,0000b849f77a49e4a4ce2b2a4ca5be3f,115,1,18.90,0,1
4,0004aac84e0df4da2b147fca70cf8255,288,1,180.00,1,3
9,0005ef4cd20d2893f0d9fbd94d3c0d97,170,1,104.90,0,2
10,0006fdc98a402fceb4eb0ee528f6a8d4,408,1,13.90,1,1
13,000949456b182f53c18b68d6babc79c1,129,1,64.89,0,1
...,...,...,...,...,...,...
93320,fff22793223fe80c97a8fd02ac5c6295,65,1,66.00,1,1
93324,fff5eb4918b2bf4b2da476788d42051c,58,1,1050.00,0,2
93328,fffa431dd3fcdefea4b1777d114144f2,303,1,64.99,0,3
93333,ffff371b4d645b6ecea244b27531430a,568,1,89.90,1,2


In [ ]:
df_review = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/time_review.csv")

In [ ]:
df_review2 = df_review.iloc[:,[2,13,14,15]]

In [ ]:
df_review2

,customer_id,review_score,review_comment_title,review_comment_message
0,9ef432eb6251297304e76186b10a928d,4,NaN,"Não testei o produto ainda, mas ele veio corre..."
1,b0830fb4747a6c6d20dea0b8c802d7ef,4,Muito boa a loja,Muito bom o produto.
2,41ce2a54c0b03bf3443c3d931a367089,5,NaN,NaN
3,f88197465ea7920adcdbec7375364d82,5,NaN,O produto foi exatamente o que eu esperava e e...
4,8ab97904e6daea8866dbdbc4fb7aad2c,5,NaN,NaN
...,...,...,...,...
96333,39bd1228ee8140590ac3aca26f2dfe00,5,NaN,NaN
96334,1fca14ff2861355f6e5f14306ff977a7,4,NaN,So uma peça que veio rachado mas tudo bem rs
96335,1aa71eb042121263aafbe80c1b562c9c,5,NaN,Foi entregue antes do prazo.
96336,b331b74b18dc79bcdf6532d51e1637c1,2,NaN,Foi entregue somente 1. Quero saber do outro p...


In [ ]:
final_df = pd.merge(df_review2, rfm_data3, on='customer_id', how='inner')

In [ ]:
rfm_data3.to_csv("/content/gdrive/My Drive/Colab Notebooks/time_rfm.csv")